In [1]:
from datetime import datetime
from distutils.util import strtobool

import pandas as pd


# Converts the contents in a .tsf file into a dataframe and returns it along with other meta-data of the dataset: frequency, horizon, whether the dataset contains missing values and whether the series have equal lengths
#
# Parameters
# full_file_path_and_name - complete .tsf file path
# replace_missing_vals_with - a term to indicate the missing values in series in the returning dataframe
# value_column_name - Any name that is preferred to have as the name of the column containing series values in the returning dataframe
def convert_tsf_to_dataframe(
    full_file_path_and_name,
    replace_missing_vals_with="NaN",
    value_column_name="series_value",
):
    col_names = []
    col_types = []
    all_data = {}
    line_count = 0
    frequency = None
    forecast_horizon = None
    contain_missing_values = None
    contain_equal_length = None
    found_data_tag = False
    found_data_section = False
    started_reading_data_section = False

    with open(full_file_path_and_name, "r", encoding="cp1252") as file:
        for line in file:
            # Strip white space from start/end of line
            line = line.strip()

            if line:
                if line.startswith("@"):  # Read meta-data
                    if not line.startswith("@data"):
                        line_content = line.split(" ")
                        if line.startswith("@attribute"):
                            if (
                                len(line_content) != 3
                            ):  # Attributes have both name and type
                                raise Exception("Invalid meta-data specification.")

                            col_names.append(line_content[1])
                            col_types.append(line_content[2])
                        else:
                            if (
                                len(line_content) != 2
                            ):  # Other meta-data have only values
                                raise Exception("Invalid meta-data specification.")

                            if line.startswith("@frequency"):
                                frequency = line_content[1]
                            elif line.startswith("@horizon"):
                                forecast_horizon = int(line_content[1])
                            elif line.startswith("@missing"):
                                contain_missing_values = bool(
                                    strtobool(line_content[1])
                                )
                            elif line.startswith("@equallength"):
                                contain_equal_length = bool(strtobool(line_content[1]))

                    else:
                        if len(col_names) == 0:
                            raise Exception(
                                "Missing attribute section. Attribute section must come before data."
                            )

                        found_data_tag = True
                elif not line.startswith("#"):
                    if len(col_names) == 0:
                        raise Exception(
                            "Missing attribute section. Attribute section must come before data."
                        )
                    elif not found_data_tag:
                        raise Exception("Missing @data tag.")
                    else:
                        if not started_reading_data_section:
                            started_reading_data_section = True
                            found_data_section = True
                            all_series = []

                            for col in col_names:
                                all_data[col] = []

                        full_info = line.split(":")

                        if len(full_info) != (len(col_names) + 1):
                            raise Exception("Missing attributes/values in series.")

                        series = full_info[len(full_info) - 1]
                        series = series.split(",")

                        if len(series) == 0:
                            raise Exception(
                                "A given series should contains a set of comma separated numeric values. At least one numeric value should be there in a series. Missing values should be indicated with ? symbol"
                            )

                        numeric_series = []

                        for val in series:
                            if val == "?":
                                numeric_series.append(replace_missing_vals_with)
                            else:
                                numeric_series.append(float(val))

                        if numeric_series.count(replace_missing_vals_with) == len(
                            numeric_series
                        ):
                            raise Exception(
                                "All series values are missing. A given series should contains a set of comma separated numeric values. At least one numeric value should be there in a series."
                            )

                        all_series.append(pd.Series(numeric_series).array)

                        for i in range(len(col_names)):
                            att_val = None
                            if col_types[i] == "numeric":
                                att_val = int(full_info[i])
                            elif col_types[i] == "string":
                                att_val = str(full_info[i])
                            elif col_types[i] == "date":
                                att_val = datetime.strptime(
                                    full_info[i], "%Y-%m-%d %H-%M-%S"
                                )
                            else:
                                raise Exception(
                                    "Invalid attribute type."
                                )  # Currently, the code supports only numeric, string and date types. Extend this as required.

                            if att_val is None:
                                raise Exception("Invalid attribute value.")
                            else:
                                all_data[col_names[i]].append(att_val)

                line_count = line_count + 1

        if line_count == 0:
            raise Exception("Empty file.")
        if len(col_names) == 0:
            raise Exception("Missing attribute section.")
        if not found_data_section:
            raise Exception("Missing series information under data section.")

        all_data[value_column_name] = all_series
        loaded_data = pd.DataFrame(all_data)

        return (
            loaded_data,
            frequency,
            forecast_horizon,
            contain_missing_values,
            contain_equal_length,
        )


# Example of usage
#loaded_data, frequency, forecast_horizon, contain_missing_values, contain_equal_length = convert_tsf_to_dataframe("TSForecasting/tsf_data/sample.tsf")

# print(loaded_data)
# print(frequency)
# print(forecast_horizon)
# print(contain_missing_values)
# print(contain_equal_length)

In [2]:
loaded_data, frequency, forecast_horizon, contain_missing_values, contain_equal_length = convert_tsf_to_dataframe("phase_1_data.tsf")

print(loaded_data)


   series_name     start_timestamp  \
0    Building0 2016-07-03 21:30:00   
1    Building1 2019-01-09 23:15:00   
2    Building3 2016-03-01 04:15:00   
3    Building4 2019-07-03 04:45:00   
4    Building5 2019-07-25 23:00:00   
5    Building6 2019-07-25 01:45:00   
6       Solar0 2020-04-25 14:00:00   
7       Solar1 2018-12-31 13:00:00   
8       Solar2 2019-06-05 14:00:00   
9       Solar3 2019-06-05 14:00:00   
10      Solar4 2019-06-05 14:00:00   
11      Solar5 2019-01-15 13:00:00   

                                         series_value  
0   [283.8, 283.8, 283.8, 606.0, 606.0, 606.0, 606...  
1   [8.1, 15.7, 22.8, 32.7, 8.1, 16.5, 24.7, 34.5,...  
2   [1321.0, 1321.0, 1321.0, 1321.0, 1293.0, 1293....  
3   [2.0, NaN, 1.0, 2.0, NaN, 2.0, NaN, NaN, 2.0, ...  
4   [30.0, 31.0, 24.0, 34.0, 30.0, 31.0, 26.0, 33....  
5   [36.8, 34.6, 34.6, 36.2, 36.2, 35.2, 35.2, 35....  
6   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
7   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
8

In [3]:
df1 = pd.DataFrame(loaded_data)
df1.series_value[0]

<PandasArray>
[283.8, 283.8, 283.8, 606.0, 606.0, 606.0, 606.0, 306.0, 306.0, 306.0,
 ...
 142.7, 142.7, 142.7,  96.9,  96.9,  96.9,  96.9,  37.4,  37.4,  37.4]
Length: 148810, dtype: object

In [4]:
df1.series_name[0]

'Building0'

In [5]:
t = pd.date_range(start ='07-03-2016 21:30:00', 
         periods = 148810 , freq ='15Min')

In [6]:
data = {'timestamp': t,
        'building0': df1.series_value[0]}

In [7]:
building0 = pd.DataFrame(data)
building0

,timestamp,building0
0,2016-07-03 21:30:00,283.8
1,2016-07-03 21:45:00,283.8
2,2016-07-03 22:00:00,283.8
3,2016-07-03 22:15:00,606
4,2016-07-03 22:30:00,606
...,...,...
148805,2020-09-30 22:45:00,96.9
148806,2020-09-30 23:00:00,96.9
148807,2020-09-30 23:15:00,37.4
148808,2020-09-30 23:30:00,37.4


In [8]:
df1

,series_name,start_timestamp,series_value
0,Building0,2016-07-03 21:30:00,"[283.8, 283.8, 283.8, 606.0, 606.0, 606.0, 606..."
1,Building1,2019-01-09 23:15:00,"[8.1, 15.7, 22.8, 32.7, 8.1, 16.5, 24.7, 34.5,..."
2,Building3,2016-03-01 04:15:00,"[1321.0, 1321.0, 1321.0, 1321.0, 1293.0, 1293...."
3,Building4,2019-07-03 04:45:00,"[2.0, NaN, 1.0, 2.0, NaN, 2.0, NaN, NaN, 2.0, ..."
4,Building5,2019-07-25 23:00:00,"[30.0, 31.0, 24.0, 34.0, 30.0, 31.0, 26.0, 33...."
5,Building6,2019-07-25 01:45:00,"[36.8, 34.6, 34.6, 36.2, 36.2, 35.2, 35.2, 35...."
6,Solar0,2020-04-25 14:00:00,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
7,Solar1,2018-12-31 13:00:00,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
8,Solar2,2019-06-05 14:00:00,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
9,Solar3,2019-06-05 14:00:00,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [9]:
df1.series_name[1]

'Building1'

In [10]:
df1.series_value[1]

<PandasArray>
[ 8.1, 15.7, 22.8, 32.7,  8.1, 16.5, 24.7, 34.5,  8.2, 16.0,
 ...
  9.2, 13.4, 17.8,  4.9,  9.3, 14.4, 18.8,  4.5,  9.6, 14.0]
Length: 60483, dtype: object

In [11]:
t1 = pd.date_range(start ='01-09-2019 23:15:00', 
         periods = 60483 , freq ='15Min')
data1 = {'timestamp': t1,
        'building1': df1.series_value[1]}
building1 = pd.DataFrame(data1)
building1

,timestamp,building1
0,2019-01-09 23:15:00,8.1
1,2019-01-09 23:30:00,15.7
2,2019-01-09 23:45:00,22.8
3,2019-01-10 00:00:00,32.7
4,2019-01-10 00:15:00,8.1
...,...,...
60478,2020-09-30 22:45:00,14.4
60479,2020-09-30 23:00:00,18.8
60480,2020-09-30 23:15:00,4.5
60481,2020-09-30 23:30:00,9.6


In [12]:
df1.series_value[2]

<PandasArray>
[1321.0, 1321.0, 1321.0, 1321.0, 1293.0, 1293.0, 1293.0, 1293.0, 1249.0,
 1249.0,
 ...
  420.0,  420.0,  420.0,  389.0,  389.0,  389.0,  389.0,  415.0,  415.0,
  415.0]
Length: 160783, dtype: object

In [13]:
t3 = pd.date_range(start ='03-01-2016 04:15:00', 
         periods = 160783 , freq ='15Min')
data3 = {'timestamp': t3,
        'building3': df1.series_value[2]}
building3 = pd.DataFrame(data3)
building3

,timestamp,building3
0,2016-03-01 04:15:00,1321
1,2016-03-01 04:30:00,1321
2,2016-03-01 04:45:00,1321
3,2016-03-01 05:00:00,1321
4,2016-03-01 05:15:00,1293
...,...,...
160778,2020-09-30 22:45:00,389
160779,2020-09-30 23:00:00,389
160780,2020-09-30 23:15:00,415
160781,2020-09-30 23:30:00,415


In [14]:
df1.series_value[3], df1.series_name[3], df1.start_timestamp[3]

(<PandasArray>
 [  2.0, 'NaN',   1.0,   2.0, 'NaN',   2.0, 'NaN', 'NaN',   2.0,   1.0,
  ...
    2.0, 'NaN',   1.0, 'NaN', 'NaN', 'NaN',   1.0, 'NaN', 'NaN', 'NaN']
 Length: 43757, dtype: object,
 'Building4',
 Timestamp('2019-07-03 04:45:00'))

In [15]:
t4 = pd.date_range(start ='07-03-2019 04:45:00', 
         periods = 43757 , freq ='15Min')
data4 = {'timestamp': t4,
        'building4': df1.series_value[3]}
building4 = pd.DataFrame(data4)
building4

,timestamp,building4
0,2019-07-03 04:45:00,2
1,2019-07-03 05:00:00,NaN
2,2019-07-03 05:15:00,1
3,2019-07-03 05:30:00,2
4,2019-07-03 05:45:00,NaN
...,...,...
43752,2020-09-30 22:45:00,NaN
43753,2020-09-30 23:00:00,1
43754,2020-09-30 23:15:00,NaN
43755,2020-09-30 23:30:00,NaN


In [16]:
df1.series_value[4], df1.series_name[4], df1.start_timestamp[4]

(<PandasArray>
 [ 30.0,  31.0,  24.0,  34.0,  30.0,  31.0,  26.0,  33.0,  28.0,  33.0,
  ...
  'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN']
 Length: 41572, dtype: object,
 'Building5',
 Timestamp('2019-07-25 23:00:00'))

In [17]:
t5 = pd.date_range(start ='07-25-2019 23:00:00', 
         periods = 41572 , freq ='15Min')
data5 = {'timestamp': t5,
        'building5': df1.series_value[4]}
building5 = pd.DataFrame(data5)
building5

,timestamp,building5
0,2019-07-25 23:00:00,30
1,2019-07-25 23:15:00,31
2,2019-07-25 23:30:00,24
3,2019-07-25 23:45:00,34
4,2019-07-26 00:00:00,30
...,...,...
41567,2020-09-30 22:45:00,NaN
41568,2020-09-30 23:00:00,NaN
41569,2020-09-30 23:15:00,NaN
41570,2020-09-30 23:30:00,NaN


In [18]:
df1.series_value[5], df1.series_name[5], df1.start_timestamp[5]

(<PandasArray>
 [36.8, 34.6, 34.6, 36.2, 36.2, 35.2, 35.2, 35.8, 35.8, 36.0,
  ...
  37.4, 37.4, 35.6, 35.6, 35.6, 35.6, 38.6, 38.6, 38.6, 38.6]
 Length: 41657, dtype: object,
 'Building6',
 Timestamp('2019-07-25 01:45:00'))

In [19]:
t6 = pd.date_range(start ='07-25-2019 01:45:00', 
         periods = 41657 , freq ='15Min')
data6 = {'timestamp': t6,
        'building6': df1.series_value[5]}
building6 = pd.DataFrame(data6)
building6

,timestamp,building6
0,2019-07-25 01:45:00,36.8
1,2019-07-25 02:00:00,34.6
2,2019-07-25 02:15:00,34.6
3,2019-07-25 02:30:00,36.2
4,2019-07-25 02:45:00,36.2
...,...,...
41652,2020-09-30 22:45:00,35.6
41653,2020-09-30 23:00:00,38.6
41654,2020-09-30 23:15:00,38.6
41655,2020-09-30 23:30:00,38.6


In [20]:
building30 = pd.merge_asof(building3,building0,on='timestamp')
building30

,timestamp,building3,building0
0,2016-03-01 04:15:00,1321,NaN
1,2016-03-01 04:30:00,1321,NaN
2,2016-03-01 04:45:00,1321,NaN
3,2016-03-01 05:00:00,1321,NaN
4,2016-03-01 05:15:00,1293,NaN
...,...,...,...
160778,2020-09-30 22:45:00,389,96.9
160779,2020-09-30 23:00:00,389,96.9
160780,2020-09-30 23:15:00,415,37.4
160781,2020-09-30 23:30:00,415,37.4


In [21]:
building301 = pd.merge_asof(building30,building1,on='timestamp')
building301

,timestamp,building3,building0,building1
0,2016-03-01 04:15:00,1321,NaN,NaN
1,2016-03-01 04:30:00,1321,NaN,NaN
2,2016-03-01 04:45:00,1321,NaN,NaN
3,2016-03-01 05:00:00,1321,NaN,NaN
4,2016-03-01 05:15:00,1293,NaN,NaN
...,...,...,...,...
160778,2020-09-30 22:45:00,389,96.9,14.4
160779,2020-09-30 23:00:00,389,96.9,18.8
160780,2020-09-30 23:15:00,415,37.4,4.5
160781,2020-09-30 23:30:00,415,37.4,9.6


In [22]:
building3014 = pd.merge_asof(building301,building4,on='timestamp')
building3014

,timestamp,building3,building0,building1,building4
0,2016-03-01 04:15:00,1321,NaN,NaN,NaN
1,2016-03-01 04:30:00,1321,NaN,NaN,NaN
2,2016-03-01 04:45:00,1321,NaN,NaN,NaN
3,2016-03-01 05:00:00,1321,NaN,NaN,NaN
4,2016-03-01 05:15:00,1293,NaN,NaN,NaN
...,...,...,...,...,...
160778,2020-09-30 22:45:00,389,96.9,14.4,NaN
160779,2020-09-30 23:00:00,389,96.9,18.8,1
160780,2020-09-30 23:15:00,415,37.4,4.5,NaN
160781,2020-09-30 23:30:00,415,37.4,9.6,NaN


In [23]:
building30145 = pd.merge_asof(building3014,building5,on='timestamp')
building30145

,timestamp,building3,building0,building1,building4,building5
0,2016-03-01 04:15:00,1321,NaN,NaN,NaN,NaN
1,2016-03-01 04:30:00,1321,NaN,NaN,NaN,NaN
2,2016-03-01 04:45:00,1321,NaN,NaN,NaN,NaN
3,2016-03-01 05:00:00,1321,NaN,NaN,NaN,NaN
4,2016-03-01 05:15:00,1293,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
160778,2020-09-30 22:45:00,389,96.9,14.4,NaN,NaN
160779,2020-09-30 23:00:00,389,96.9,18.8,1,NaN
160780,2020-09-30 23:15:00,415,37.4,4.5,NaN,NaN
160781,2020-09-30 23:30:00,415,37.4,9.6,NaN,NaN


In [24]:
buildingvalues = pd.merge_asof(building30145,building6,on='timestamp')
buildingvalues

,timestamp,building3,building0,building1,building4,building5,building6
0,2016-03-01 04:15:00,1321,NaN,NaN,NaN,NaN,NaN
1,2016-03-01 04:30:00,1321,NaN,NaN,NaN,NaN,NaN
2,2016-03-01 04:45:00,1321,NaN,NaN,NaN,NaN,NaN
3,2016-03-01 05:00:00,1321,NaN,NaN,NaN,NaN,NaN
4,2016-03-01 05:15:00,1293,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
160778,2020-09-30 22:45:00,389,96.9,14.4,NaN,NaN,35.6
160779,2020-09-30 23:00:00,389,96.9,18.8,1,NaN,38.6
160780,2020-09-30 23:15:00,415,37.4,4.5,NaN,NaN,38.6
160781,2020-09-30 23:30:00,415,37.4,9.6,NaN,NaN,38.6
